In [1]:
import pandas as pd

In [2]:
content = pd.read_csv("content.csv", index_col=0)

In [3]:
content.head()

,Title,Description,IMdb_rating,Director,Genres,Img_url,Writers,Cast,IMdb_link
0,The Shawshank Redemption,"Over the course of several years, two convicts...",9.3,139,[18],https://m.media-amazon.com/images/M/MV5BNDE3OD...,"[441, 444]","[432, 347, 537]",/title/tt0111161/?ref_=chttp_t_1
1,The Godfather,"Don Vito Corleone, head of a mafia family, dec...",9.2,151,"[12, 18]",https://m.media-amazon.com/images/M/MV5BM2MyNj...,"[68, 221]","[361, 279, 382]",/title/tt0068646/?ref_=chttp_t_2
2,The Dark Knight,When the menace known as the Joker wreaks havo...,9.0,121,"[6, 12, 18]",https://m.media-amazon.com/images/M/MV5BMTMxNT...,"[39, 27, 184]","[387, 68, 201]",/title/tt0468569/?ref_=chttp_t_3
3,The Godfather Part II,The early life and career of Vito Corleone in ...,9.0,151,"[12, 18]",https://m.media-amazon.com/images/M/MV5BMWMwMG...,"[465, 65]","[279, 365, 19]",/title/tt0071562/?ref_=chttp_t_4
4,12 Angry Men,The jury in a New York City murder trial is fr...,9.0,43,"[12, 18]",https://m.media-amazon.com/images/M/MV5BMWU4N2...,[93],"[546, 386, 175]",/title/tt0050083/?ref_=chttp_t_5


In [4]:
content.dtypes

Title           object
Description     object
IMdb_rating    float64
Director         int64
Genres          object
Img_url         object
Writers         object
Cast            object
IMdb_link       object
dtype: object

In [5]:
del content["Img_url"]

In [6]:
del content["IMdb_link"]
del content["Director"]

In [7]:
content.dtypes

Title           object
Description     object
IMdb_rating    float64
Genres          object
Writers         object
Cast            object
dtype: object

In [8]:
import json

In [9]:
del content["Description"]

In [10]:
content.head()

,Title,IMdb_rating,Genres,Writers,Cast
0,The Shawshank Redemption,9.3,[18],"[441, 444]","[432, 347, 537]"
1,The Godfather,9.2,"[12, 18]","[68, 221]","[361, 279, 382]"
2,The Dark Knight,9.0,"[6, 12, 18]","[39, 27, 184]","[387, 68, 201]"
3,The Godfather Part II,9.0,"[12, 18]","[465, 65]","[279, 365, 19]"
4,12 Angry Men,9.0,"[12, 18]",[93],"[546, 386, 175]"


In [11]:
content["Genres"] = content["Genres"].apply(json.loads)
del content["Writers"]
del content["Cast"]

In [12]:
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MultiLabelBinarizer

In [13]:
mlb = MultiLabelBinarizer()
mlb_transform = pd.DataFrame(mlb.fit_transform(content['Genres']), columns=mlb.classes_)

In [14]:
features = ['IMdb_rating']
X = pd.concat([content[features],mlb_transform],axis=1)
X.columns = X.columns.astype(str)
X.head()

,IMdb_rating,0,1,2,3,4,5,6,7,8,...,11,12,13,14,15,16,17,18,19,20
0,9.3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,9.2,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
2,9.0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,1,0,0
3,9.0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
4,9.0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0


In [15]:
n = NearestNeighbors(n_neighbors=5, algorithm="brute").fit(X)

In [16]:
def show_similar_movies(movie_id):
    distances, movie_ids = n.kneighbors([X.iloc[movie_id]])
    print(f"Here are similar movies to {content.iloc[movie_id]["Title"]}")
    for i in range(len(movie_ids)):
        print(content.iloc[movie_ids[i][1:]])

In [20]:
show_similar_movies(36)

Here are similar movies to Spider-Man: Across the Spider-Verse
                                                 Title  IMdb_rating  \
64                   Spider-Man: Into the Spider-Verse          8.4   
81                                   Princess Mononoke          8.3   
249  Demon Slayer: Kimetsu no Yaiba - The Movie: Mu...          8.2   
201                           How to Train Your Dragon          8.1   

         Genres  
64   [20, 6, 9]  
81   [20, 6, 9]  
249  [20, 6, 9]  
201  [20, 6, 9]  


C:\Users\SSSha\Desktop\Projects\Recommender\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
